## Multiple SceneObjects

Test fixed and tracking sceneObjects multiple on teh same scene.

Try appendtoScene too.




In [2]:
try:
    from bifacial_radiance import RadianceObj, AnalysisObj
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
import numpy as np

testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo2'



In [7]:
# GEnerating Standard FIXED SETUP
demo = RadianceObj("MultipleObj", path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62)
epwfile = demo.getEPW(lat = 59.85, lon = 17.63)    # Coordinates for UPPSALA University
metdata = demo.readEPW(epwfile) 
fullYear = True 
#demo.genCumSky(demo.epwfile) # entire year.
demo.gendaylit(metdata,4020)  # Noon, June 17th , for visualization only!
module_type = 'Prism Solar Bi60 landscape' 
demo.makeModule(name=module_type,y=1,x=1.7,bifi = 0.90, xgap = 0.2) # this module is placed in "landscape"

# Notice that the tilt, although it's 0 relative to the roof, it is 15 relative to the ground, so we must
# use that 15 degree as tilt.
# We need to add some clearance so that it ends up at the right distance from the "roof" surface we are going to append
# afterwards. For a roof of 50.1 x 6.8m at 15 degrees, the center clearance is going to be (6.8/2) *sin( radians(15 )) = ~-.87m
# We are going to be placing the origin of this row a little bit below the actual center of the roof, so you'll have to 
# calculate exactly what clearance to use yourself. I'm ust eyeballing the value now:


# So... I just realized we have an issue with the pitch when there is only 1 row being simulated. Setting the pitch to almost 0.
sceneDict = {'tilt':15,'pitch':0.01,'clearance_height':2,'azimuth':131, 'nMods': 8, 'nRows': 1, 'originx': 0, 'originy': 0, 'appendRadfile':True} 
sceneObj1 = demo.makeScene(module_type,sceneDict)  


path = C:\Users\sayala\Documents\RadianceScenes\Demo2
Getting weather file: SWE_Stockholm.Arlanda.024600_IWEC.epw
 ... OK!


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\main.py:2410: pvlibDeprecationWarning: The get_sun_rise_set_transit function was deprecated in pvlib 0.6.1 and will be removed in 0.7. Use sun_rise_set_transit_spa instead.
  sunup= pvlib.irradiance.solarposition.get_sun_rise_set_transit(datetimetz, lat, lon) #only for pvlib <0.6.1



Module Name: Prism_Solar_Bi60_landscape
REWRITING pre-existing module file. 
Module Prism Solar Bi60 landscape successfully created


In [ ]:
# Adding the modules on portrait. This are going to be coincident with the origin (0,0)
module_type2 = 'Prism Solar Bi60 portrait' 
demo.makeModule(name=module_type2,y=1.7,x=1,bifi = 0.90) # this module is placed in "landscape"
sceneDict2 = {'tilt':45,'pitch':0.01,'clearance_height':3,'azimuth':131, 'nMods': 16, 'nRows': 1, 'originx': 0, 'originy': 0, 'appendRadfile':True} 
sceneObj2 = demo.makeScene(module_type2,sceneDict2)  
octfile = demo.makeOct(demo.getfilelist()) 


In [8]:
octfile = demo.makeOct(demo.getfilelist()) 



Created MultipleObj.oct


### At this point you should be able to go into a command window (cmd.exe) and check the geometry (first navigate to the foldre where the OCT file is saved). You should see the two rows of panels only.

Example:

### rvu -vf views\front.vp -e .01 MultipleObj.oct

# Adding the "Roof" 

In [ ]:
# This geometry is created very simply and rotated CW from S = 180. 
name='inclinedRoofGround'
size_roof_x = 20 # 50.1 # m
size_roof_y=6.8 # m
roof_azimuth = 49 #  this is the 41 deg... yeah I need to fix this at some point. But do the math.
roof_tilt = 15
text='! genbox black inclinedRoof {} {} 0 | xform -t {} 0 0 -rx {} -t 0 {} 0 -rz {}'.format(size_roof_x, size_roof_y, -size_roof_x/2.0, roof_tilt, -size_roof_y/2.0, roof_azimuth)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(radfile=sceneObj2.radfiles, customObject=customObject, text = '!xform -rz 0')

octfile = demo.makeOct(demo.getfilelist()) 

## Adding a marker at 0,0 just for sanity check ~ I would recommend not running this cell when you are getting actual results

In [9]:
# Add marker for 0,0 -- This is for S
name='Marker'
text='! genbox black marker 0.05 0.05 5 | xform -t 0 0 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(radfile=sceneObj2.radfiles, customObject=customObject, text = '!xform -rz 0')

octfile = demo.makeOct(demo.getfilelist()) 


Custom Object Name objects\Marker.rad

!xform -rz 0 objects\Marker.rad
Created MultipleObj.oct


### If you check your geometry again here, you sohuld see the  2 rows of modules, one in landscape one in portrait, the roof, and the marker (if you added it) 

rvu -vf views\front.vp -e .01 MultipleObj.oct

# ANALYSIS

You can query modules either on SceneObj1 (the landscape ones) or SceneObj2 (the portrait ones) :) 

Also, so far we are doing only 1 hour (gendaylit), but you can use genCumSky. 
*(When you use gencumsky the viewer rvu gets saturated and you can't see, so I recommend running gendaylit first, testing, and then doing the same procedure but with gencumsky for yearly results)

In [ ]:
# sceneObj saves the information of the scene, so analysis cna calculate the sensor locatoin.
sceneObj2.sceneDict

In [ ]:
# Just printing values :)
sceneObj1.sceneDict

In [ ]:
# You can continue with the 1 hour analysis as befor,e or you can do a yearly one instead:
#demo.genCumSky(demo.epwfile) # entire year.

analysis = AnalysisObj(octfile, demo.basename)  
frontscan, backscan = analysis.moduleAnalysis(sceneObj1)
frontdict, backdict = analysis.analysis(octfile, "FirstObj", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f ' %( np.mean(analysis.Wm2Back) / np.mean(analysis.Wm2Front)) )

In [ ]:
# Sanity check for first obect. Since we didn't pass any desired module, it should grab the center module of the center row (rounding down)
# somethibgn like row 0, module 4 indexed at 0, a3.0.a0.PVmodule.....""
print (frontdict['x'])
print ("")
print (frontdict['y'])
print ("")
print (frontdict['mattype'])

In [ ]:
#You can also analyse the other SceneObject. And remember you cna specify which module/row you want (although you only have 1 row atm)

analysis = AnalysisObj(octfile, demo.basename)  
frontscan, backscan = analysis.moduleAnalysis(sceneObj2, rowWanted = 1, modWanted = 10)
frontdict2, backdict2 = analysis.analysis(octfile, "SecondObj", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f ' %( np.mean(analysis.Wm2Back) / np.mean(analysis.Wm2Front)) )

In [ ]:
# Sanity check for second object. 

print (frontdict2['x'])
print ("")
print (frontdict2['y'])
print ("")
print (frontdict2['mattype'])


In [ ]:
# turns out this is wrong... there is a bug in the code since I last ran this type of simulation.
# I'll try to fish it out but it might take me a couple days.